# IV - Parameter Sweep
In this notebook we will be running a simple parameter sweep on the model we have. We will then pull the results of our sweep and based on the results of our sweep pull the best performing model from blob.

* [Setup](#section1)
* [Configure job](#section2)
* [Submit job](#section3)
* [Check results](#section4)
* [Download best model](#section5)
* [Delete job](#section6)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [29]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [30]:
shipyard

Usage: shipyard.py [OPTIONS] COMMAND [ARGS]...

  Batch Shipyard: Provision and Execute Docker Workloads on Azure Batch

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cert      Certificate actions
  data      Data actions
  fs        Filesystem in Azure actions
  jobs      Jobs actions
  keyvault  KeyVault actions
  misc      Miscellaneous actions
  pool      Pool actions
  storage   Storage actions


Read in the account information we saved earlier

In [31]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

storage_account_key = account_info['storage_account_key']
storage_account_name = account_info['storage_account_name']
IMAGE_NAME = account_info['IMAGE_NAME']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>

## Configure Job

As in the previous job we ran on a single node we will be running the job on GPU enabled nodes. The difference here is that depending on the number of combinations we will be creating the same number of tasks. Each task will have a different set of parmeters that we will be passing to our model training script. This parameters effect the training of the model and in the end the performance of the model. The model and results of its evaluation are recorded and stored on the node. At the end of the task the results are pulled into the specified storage container..

In [32]:
from copy import copy
import os
import random
from sklearn.grid_search import ParameterGrid
from toolz import curry, pipe

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

def compose_command(num_convolution_layers, minibatch_size, max_epochs=30):
    cmd_str = ' '.join(("source /cntk/activate-cntk;",
                        "python -u ConvNet_CIFAR10.py",
                        "--datadir $AZ_BATCH_NODE_SHARED_DIR/data",
                        "--num_convolution_layers {num_convolution_layers}",
                        "--minibatch_size {minibatch_size}",
                        "--max_epochs {max_epochs}")).format(num_convolution_layers=num_convolution_layers,
                                                             minibatch_size=minibatch_size,
                                                             max_epochs=max_epochs)
    return 'bash -c "{}"'.format(cmd_str)

@curry
def append_parameter(param_name, param_value, data_dict):
    data_dict[param_name]=param_value
    return data_dict

def task_generator(parameters):
    for params in ParameterGrid(parameters):
        yield pipe(copy(_TASK_TEMPLATE),
                   append_parameter('command', compose_command(**params)))

Generate the `jobs.json` configuration file

In [33]:
_TASK_TEMPLATE = {
    "image": IMAGE_NAME,
    "remove_container_after_exit": True,
    "gpu": True,
    "resource_files": [
            {
                "file_path": "ConvNet_CIFAR10.py",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/ConvNet_CIFAR10.py",
                "file_mode":'0777'
            }
    ],
    "output_data": {
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": "output",
                "source": "$AZ_BATCH_TASK_DIR/wd/Models"
            },
        ]
    },
}

For the purposes of the workshop, we are constraining the parameter search space to just 3 final combinations (4 will be generated but we will remove the first one such that we only need to wait for "one round" of processing. In other words, since we have 3 total compute nodes in the pool and 3 tasks, we only have to wait for "one round."

In [35]:
parameters = {
    "num_convolution_layers": [2, 3],  # this could be expanded to [2, 3, 4] for example
    "minibatch_size": [32, 64]         # this could be expanded to [32, 64, 128] for example
}

In [36]:
JOB_ID = 'cntk-parametricsweep-job'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": list(task_generator(parameters))    
        }
    ]
}

# for purposes of expediency in the workshop, we'll remove one of the tasks to
# make 3 total to match the number of compute nodes in our pool
del jobs['job_specifications'][0]['tasks'][0]

print('number of tasks for parametric sweep {}: {}'.format(JOB_ID, len(jobs['job_specifications'][0]['tasks'])))

number of tasks for parametric sweep cntk-parametricsweep-job: 3


In [37]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

{
    "job_specifications": [
        {
            "id": "cntk-parametricsweep-job", 
            "tasks": [
                {
                    "command": "bash -c \"source /cntk/activate-cntk; python -u ConvNet_CIFAR10.py --datadir $AZ_BATCH_NODE_SHARED_DIR/data --num_convolution_layers 3 --minibatch_size 32 --max_epochs 30\"", 
                    "gpu": true, 
                    "image": "microsoft/cntk:2.0-gpu-python3.5-cuda8.0-cudnn5.1", 
                    "output_data": {
                        "azure_storage": [
                            {
                                "container": "output", 
                                "source": "$AZ_BATCH_TASK_DIR/wd/Models", 
                                "storage_account_settings": "mystorageaccount"
                            }
                        ]
                    }, 
                    "remove_container_after_exit": true, 
                    "resource_files": [
                        {
                       

<a id='section3'></a>

## Submit job
Check that everything is ok with our pool before we submit our jobs

In [38]:
shipyard pool listnodes

2017-06-21 15:17:54,736 DEBUG - listing nodes for pool gpupool
2017-06-21 15:17:55,023 INFO - node_id=tvm-4283973576_1-20170621t143056z [state=ComputeNodeState.idle start_task_exit_code=0 scheduling_state=SchedulingState.enabled ip_address=10.0.0.4 vm_size=standard_nc6 dedicated=True total_tasks_run=2 running_tasks_count=0 total_tasks_succeeded=2]
2017-06-21 15:17:55,023 INFO - node_id=tvm-4283973576_2-20170621t143056z [state=ComputeNodeState.idle start_task_exit_code=0 scheduling_state=SchedulingState.enabled ip_address=10.0.0.6 vm_size=standard_nc6 dedicated=True total_tasks_run=0 running_tasks_count=0 total_tasks_succeeded=0]
2017-06-21 15:17:55,023 INFO - node_id=tvm-4283973576_3-20170621t143056z [state=ComputeNodeState.idle start_task_exit_code=0 scheduling_state=SchedulingState.enabled ip_address=10.0.0.5 vm_size=standard_nc6 dedicated=True total_tasks_run=0 running_tasks_count=0 total_tasks_succeeded=0]


Now that we have confirmed everything is working we can execute our job using the command below. 

In [39]:
shipyard jobs add

2017-06-21 15:18:01,986 INFO - Adding job cntk-parametricsweep-job to pool gpupool
2017-06-21 15:18:02,621 INFO - uploading file /tmp/tmpjv34ca as u'shipyardtaskrf-cntk-parametricsweep-job/dockertask-00000.shipyard.envlist'
2017-06-21 15:18:03,086 INFO - uploading file /tmp/tmpdrlMhz as u'shipyardtaskrf-cntk-parametricsweep-job/dockertask-00001.shipyard.envlist'
2017-06-21 15:18:03,587 INFO - uploading file /tmp/tmpaXR1Qj as u'shipyardtaskrf-cntk-parametricsweep-job/dockertask-00002.shipyard.envlist'
2017-06-21 15:18:03,805 DEBUG - submitting 3 tasks (0 -> 2) to job cntk-parametricsweep-job
2017-06-21 15:18:04,055 INFO - submitted all 3 tasks to job cntk-parametricsweep-job


Using the command below we can check the status of our job. Only after all tasks have an exit code can we continue with the notebook. Please keep re-running the cell below periodically until you see that all tasks show completed state with an exit code. Continuing on with the notebook without all tasks in the job completing their training execution will result in failure in subsequent cells.

You can also view the **heatmap** of this pool on [Azure Portal](https://portal.azure.com) to monitor the progress of this job on the compute nodes under your Batch account.

In [40]:
shipyard jobs listtasks --jobid $JOB_ID

2017-06-21 15:19:53,954 INFO - job_id=cntk-parametricsweep-job task_id=dockertask-00000 [state=TaskState.running max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-4283973576_3-20170621t143056z start_time=2017-06-21 15:18:06.619468+00:00 end_time=None duration=n/a exit_code=None]
2017-06-21 15:19:53,954 INFO - job_id=cntk-parametricsweep-job task_id=dockertask-00001 [state=TaskState.running max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-4283973576_2-20170621t143056z start_time=2017-06-21 15:18:05.822523+00:00 end_time=None duration=n/a exit_code=None]
2017-06-21 15:19:53,955 INFO - job_id=cntk-parametricsweep-job task_id=dockertask-00002 [state=TaskState.running max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-4283973576_1-20170621t143056z start_time=2017-06-21 15:18:05.827206+00:00 end_time=None duration=n/a exit_code=None]


<a id='section4'></a>

# Check results
The results of our parameter search should now be saved to our output container.

**Note:** You will encounter errors if you did not wait for all tasks to complete with an exit code in the previous cell.

First let's alias `blobxfer` to aid in downloading our blobs. We will aggregate our results in the `MODELS_DIR`.

In [21]:
%alias blobxfer python -m blobxfer

MODELS_DIR = 'psmodels'

In [22]:
blobxfer $storage_account_name output $MODELS_DIR --remoteresource . --download --include "*_$JOB_ID/model_results.json" --storageaccountkey $storage_account_key

 azure blobxfer parameters [v0.12.1]
             platform: Linux-4.4.0-75-generic-x86_64-with-debian-stretch-sid
   python interpreter: CPython 2.7.11
     package versions: az.common=1.1.5 az.sml=0.20.5 az.stor=0.34.2 crypt=1.9 req=2.18.1
      subscription id: None
      management cert: None
   transfer direction: Azure->local
       local resource: psmodels
      include pattern: *_cntk-parametricsweep-job/model_results.json
      remote resource: .
   max num of workers: 24
              timeout: None
      storage account: batche2914cdbst
              use SAS: False
  upload as page blob: False
  auto vhd->page blob: False
 upload to file share: False
 container/share name: output
  container/share URI: https://batche2914cdbst.blob.core.windows.net/output
    compute block MD5: False
     compute file MD5: True
    skip on MD5 match: True
   chunk size (bytes): 4194304
     create container: True
  keep mismatched MD5: False
     recursive if dir: True
component strip on up: 1


Now we will combine all of the `model_results.json` files into one dictionary for analysis.

In [23]:
def scandir(basedir):
    for root, dirs, files in os.walk(basedir):
        for f in files:
            yield os.path.join(root, f) 

results_dict = {}
for model in scandir(MODELS_DIR):
    if not model.endswith('.json'):
        continue
    key = model.split(os.sep)[1]
    results_dict[key] = read_json(model)
    
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "4fe7fb06_dockertask-00000_cntk-parametricsweep-job": {
        "parameters": {
            "max_epochs": 30, 
            "minibatch_size": 32, 
            "num_convolution_layers": 3
        }, 
        "test_metric": 0.2188
    }, 
    "586269dc_dockertask-00001_cntk-parametricsweep-job": {
        "parameters": {
            "max_epochs": 30, 
            "minibatch_size": 64, 
            "num_convolution_layers": 2
        }, 
        "test_metric": 0.2284
    }, 
    "b11855a9_dockertask-00002_cntk-parametricsweep-job": {
        "parameters": {
            "max_epochs": 30, 
            "minibatch_size": 64, 
            "num_convolution_layers": 3
        }, 
        "test_metric": 0.2311
    }
}


From the aggregated results dictionary, we select the one with the smallest error:

In [24]:
tuple_min_error = min(results_dict.iteritems(), key=lambda x: x[1]['test_metric'])
configuration_with_min_error = tuple_min_error[0]
print('task with smallest error: {} ({})'.format(configuration_with_min_error, tuple_min_error[1]['test_metric']))

task with smallest error: 4fe7fb06_dockertask-00000_cntk-parametricsweep-job (0.2188)


<a id='section5'></a>

## Download best model
Now we'll download the corresponding best performing model.

In [25]:
MODEL_NAME = 'ConvNet_CIFAR10_model.dnn'
BEST_MODEL_BLOB_NAME = '{}/{}'.format(configuration_with_min_error, MODEL_NAME)
print(BEST_MODEL_BLOB_NAME)

4fe7fb06_dockertask-00000_cntk-parametricsweep-job/ConvNet_CIFAR10_model.dnn


In [26]:
blobxfer $storage_account_name output $MODELS_DIR --remoteresource $BEST_MODEL_BLOB_NAME --download --storageaccountkey $storage_account_key

 azure blobxfer parameters [v0.12.1]
             platform: Linux-4.4.0-75-generic-x86_64-with-debian-stretch-sid
   python interpreter: CPython 2.7.11
     package versions: az.common=1.1.5 az.sml=0.20.5 az.stor=0.34.2 crypt=1.9 req=2.18.1
      subscription id: None
      management cert: None
   transfer direction: Azure->local
       local resource: psmodels
      include pattern: None
      remote resource: 4fe7fb06_dockertask-00000_cntk-parametricsweep-job/ConvNet_CIFAR10_model.dnn
   max num of workers: 24
              timeout: None
      storage account: batche2914cdbst
              use SAS: False
  upload as page blob: False
  auto vhd->page blob: False
 upload to file share: False
 container/share name: output
  container/share URI: https://batche2914cdbst.blob.core.windows.net/output
    compute block MD5: False
     compute file MD5: True
    skip on MD5 match: True
   chunk size (bytes): 4194304
     create container: True
  keep mismatched MD5: False
     recursive if d

In [27]:
!mv $MODELS_DIR/$configuration_with_min_error/$MODEL_NAME $MODELS_DIR
!rm -rf $MODELS_DIR/*_$JOB_ID  # optionally remove all of the temporary result json directories/files
!ls -alF $MODELS_DIR

total 1920
drwxr-xr-x  2 nbuser nbuser    4096 Jun 21 09:59 ./
drwx------ 11 nbuser nbuser    4096 Jun 21 09:45 ../
-rw-r--r--  1 nbuser nbuser 1957354 Jun 21 09:59 ConvNet_CIFAR10_model.dnn


The best model file (`ConvNet_CIFAR10_model.dnn`) is now ready for use.

**Note:** We could have created a Batch task that did the model selection for us using task dependencies. The model selection task would be dependent upon all of the parametric sweep training tasks and would only run after those tasks complete successfully. The Batch task could then proceed with the logic above.

Please see the advanced notebook that shows how this is accomplished: [Automatic Model Selection from Parametric Sweep with Task Dependencies](06_Advanced_Auto_Model_Selection.ipynb)

<a id='section6'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [41]:
shipyard jobs del -y --termtasks --wait

2017-06-21 15:27:21,415 INFO - Deleting job: cntk-ps-as-job
2017-06-21 15:27:21,415 DEBUG - disabling job cntk-ps-as-job first due to task termination
2017-06-21 15:27:21,833 ERROR - cntk-ps-as-job job does not exist


## Next Steps
You can proceed to the [Notebook: Clean Up](05_Clean_Up.ipynb) 